<a href="https://colab.research.google.com/github/kslim1025/TF_Keras_RNN/blob/master/RNN_KoreanLang_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# MNIST 문자열 분석으로 유명한 데이터셋이 패션에 관한 데이터 세트를 만듬 그게 FashionMINIST 데이터셋이다.
# 데이터 이미지가 0에서 255까지 값을 가지는 28x28이미지라는 것을 확인가능
# 정답이 되는 라벨을 확인하기 위해 print를 붙여서  확인
# 외부 데이터를 이용한 정제과정
# ctrl+enter를 사용한 런타임가능
# !nvidia-smi : 어떤 GPU를 사용하는지 확인가능한 명령어 
#

try:
  # %tensorflow_version only exists in Colab
  %tensorflow_version 2.x
except Exception:
  pass

# 넘파이는 수학과 과학 연산에 특화된 파이썬 모듈로 딥러닝에서도 유용하게 사용된다.

import numpy as np;
import tensorflow as tf;
import pandas as pd;
import matplotlib.pyplot as plt;
import math;
import re;
################################################################################
!nvidia-smi
       
#+-----------------------------------------------------------------------------+
#| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
#|-------------------------------+----------------------+----------------------+
#| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
#| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
#|                               |                      |               MIG M. |
#|===============================+======================+======================|
#|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
#| N/A   51C    P0    35W / 250W |   1581MiB / 16280MiB |      0%      Default |
#|                               |                      |                 ERR! |
#+-------------------------------+----------------------+----------------------+
#                                                                               
#+-----------------------------------------------------------------------------+
#| Processes:                                                                  |
#|  GPU   GI   CI        PID   Type   Process name                  GPU Memory |
#|        ID   ID                                                   Usage      |
#|=============================================================================|
#|  No running processes found                                                 |
#+-----------------------------------------------------------------------------+

################################################################################

# 한글 원본 텍스트를자소 단위와 단어 단위로 나눠서 순환 신경망으로 생성
# 이번에 작업할때 사용할 원본 텍스트는 국사 편찬위원회에서 제공하는 조선왕조실록 국문 번역본을 사용

path_to_file = tf.keras.utils.get_file('input.txt', 'http://bit.ly/2Mc3SOV')

#데이터를 메모리에 불러오기, 인코딩 형식은 utf-8로 지정
train_text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

#텍스트가 총 몇 자인지를 확인
print('텍스트의 길이 : {} charaters'.format(len(train_text)))
print()

#처음에는 300자를 임시로 확인
print(train_text[:300])

#훈련 데이터입력 정제
# from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py

def clean_str(string):
  string = re.sub(r"[^가-힣A-Za-z0-9(),!?'\']"," ",string)
  string = re.sub(r"\'s", "\'s", string)
  string = re.sub(r"\'ve","\'ve",string)
  string = re.sub(r"n\'t", "n\'t",string)
  string = re.sub(r"\'re","\'re",string)
  string = re.sub(r"\'d","\'d",string)
  string = re.sub(r"\'ll","\'ll",string)
  string = re.sub(r",",",",string)
  string = re.sub(r"!"," ! ",string)
  string = re.sub(r"\(", " \( ",string)
  string = re.sub(r"\)", " \) ",string)
  string = re.sub(r"\?", " \? ",string)
  string = re.sub(r"\?", " \? ",string)
  string = re.sub(r"\s{2,}"," ",string)
  string = re.sub(r"\s{2,}","\'",string)
  string = re.sub(r"\'","",string)
  
  return string

train_text = train_text.split('\n')
train_text = [clean_str(sentence) for sentence in train_text]
train_text_X = []

for sentence in train_text:
  train_text_X.extend(sentence.split(' '))
  train_text_X.append('\n')

train_text_X = [word for word in train_text_X if word != '']

print(train_text_X[:20])

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.

텍스트의 길이 : 26265493 charaters

﻿태조 이성계 선대의 가계. 목조 이안사가 전주에서 삼척·의주를 거쳐 알동에 정착하다 
태조 강헌 지인 계운 성문 신무 대왕(太祖康獻至仁啓運聖文神武大王)의 성은 이씨(李氏)요, 휘(諱)는 단(旦)이요, 자(字)는 군진(君晉)이다. 그전의 휘(諱)는 이성계(李成桂)요, 호(號)는 송헌(松軒)이다. 전주(全州)의 대성(大姓)이다. 사공(司空) 휘(諱) 이한(李翰)이 신라(新羅)에 벼슬하여 태종왕(太宗王)001) 의 10대(代) 손자인 군윤(軍尹) 김은의(金殷義)의 딸에게 장가들어 시중(侍中) 휘(諱) 이자연(李自延)을 낳았다. 시중이 
['태조', '이성계', '선대의', '가계', '목조', '이안사가', '전주에서', '삼척', '의주를', '거쳐', '알동에', '정착하다', '\n', '태조', '강헌', '지인', '계운', '성문', '신무', '대왕']
